<a href="https://colab.research.google.com/github/ajits-github/Advanced_Python/blob/main/Advanced_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GIL and Mutex

GIL is a lock based concept which is of more concerns for the tasks related to CPU and a less of a limitation when it comes to I/O bound tasks. This is because I/O bound tasks generally involve lots of waiting. For example, reading or wirting the files, or network communications or interacting with a database etc., where one thread has to wait and hence GIL releases the lock and the service to be used by other threads. These tasks are limited by the speed of the input/output operations and typically involve more waiting than actual computation.

About the GIL and its usefulness, it's important to understand that the GIL is a trade-off. While the GIL restricts the execution of Python bytecode to a single thread, it also simplifies memory management, prevents race conditions related to memory access, and makes Python code more thread-safe by default.

Mutex is the accronym for mutual exclusion. In python, mutex are those elements which comes in the play when something is shared between different resources. Along with GIL (Global Interpreter Lock), this ensures that only one thread is accessing the object at a time.

Here's a simple example in Python using the threading module to illustrate how a mutex works:

In [ ]:
import threading

# Shared resource
shared_counter = 0

# Mutex for controlling access to shared_counter
mutex = threading.Lock()

# Function that increments the shared_counter
def increment_counter():
    global shared_counter
    for _ in range(100000):
        # Acquire the mutex
        mutex.acquire()
        shared_counter += 1
        # Release the mutex
        mutex.release()

# Create two threads that increment the counter
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)

# Start the threads
thread1.start()
thread2.start()

# Wait for the threads to finish
thread1.join()
thread2.join()

print("Final value of shared_counter:", shared_counter)


Final value of shared_counter: 200000


In this example, the threading.Lock() object (mutex) is used to protect access to the shared_counter. The increment_counter function acquires the mutex before modifying the counter and releases it afterward. This ensures that only one thread can modify the counter at a time, preventing data corruption due to concurrent access.

Mutexes are a fundamental concept in concurrent programming, and they play a crucial role in ensuring the correctness and consistency of shared data in multi-threaded or multi-process applications.

**If you don't use a mutex** to protect the shared_counter in a multi-threaded environment, you might encounter a race condition. A race condition occurs when multiple threads access shared resources concurrently, and the final state of the resource depends on the timing and order of the threads' execution. In this case, without proper synchronization (such as a mutex), the following issues can arise:

**Data Corruption:** Since multiple threads can read and modify the shared_counter without coordination, they may read an outdated value, leading to incorrect updates. This can result in an inconsistent or incorrect final value for shared_counter.

**Lost Updates:**Threads can overwrite each other's updates if they're not properly synchronized. For example, if two threads simultaneously read the counter's value as 5 and increment it to 6, both threads will update the counter to 6, effectively losing one update.

**Inconsistent State:** Due to the unpredictable order of thread execution, the shared_counter might end up with an arbitrary value, leading to an inconsistent or unpredictable state.

To avoid these issues, using a mutex (like the threading.Lock() in the example) ensures that only one thread at a time can access and modify the shared resource, preventing race conditions and ensuring data integrity.

In [ ]:
import threading

# A shared list
shared_list = []

# Mutex for controlling access to shared_list
mutex = threading.Lock()

# Function that appends to the shared list
def append_to_list():
    global shared_list
    for _ in range(100000):
        # Acquire the mutex
        mutex.acquire()
        shared_list.append(threading.current_thread().name)
        # Release the mutex
        mutex.release()

# Create two threads that append to the list
thread1 = threading.Thread(target=append_to_list, name="Thread 1")
thread2 = threading.Thread(target=append_to_list, name="Thread 2")

# Start the threads
thread1.start()
thread2.start()

# Wait for the threads to finish
thread1.join()
thread2.join()

print("Shared list:", shared_list)


In this example, the GIL ensures that only one thread can execute the Python bytecode at a time, even though both threads are trying to append to the shared_list simultaneously. As a result, the threads acquire and release the mutex sequentially, effectively serializing access to the shared_list, and the final shared_list will contain only elements from one of the threads, demonstrating how the GIL affects multi-threaded CPU-bound operations.

Regarding the use of the **global keyword** in Python, it's used to indicate that a variable inside a function should refer to a variable defined in the global scope rather than creating a new local variable. It's commonly used when you want to modify or access a global variable from within a function. However, using global variables can lead to code that's harder to reason about, maintain, and test, as they introduce hidden dependencies and can make functions less modular.

It's generally recommended to avoid excessive use of global variables. Instead, it's better to pass variables as arguments to functions and return values from functions. This promotes better encapsulation and makes your code more self-contained and easier to understand.



---



**Mutexes** in Python are implemented using the threading.Lock() class from the threading module. Here are a few examples of using mutexes to protect shared resources in multi-threaded environments:

**Using Mutex with Context Manager:**

In [ ]:
import threading

shared_list = []
mutex = threading.Lock()

def append_to_list():
  global shared_list
  for _ in range(100000):
    with mutex:
      shared_list.append(threading.current_thread().name)

Thread1 = threading.Thread(target=append_to_list, name="Thread 1")
Thread2 = threading.Thread(target=append_to_list, name="Thread 2")

Thread1.start()
Thread2.start()

Thread1.join()
Thread2.join()

print("shared list", shared_list)


**Using Mutex to Protect a Shared Dictionary:**

In [ ]:
import threading

# Shared dictionary
shared_dict = {}

# Mutex for controlling access to shared_dict
mutex = threading.Lock()

# Function that adds key-value pairs to the shared dictionary
def add_to_dict(key, value):
    with mutex:
        shared_dict[key] = value

# Create two threads that modify the dictionary
thread1 = threading.Thread(target=add_to_dict, args=("key1", "value1"))
thread2 = threading.Thread(target=add_to_dict, args=("key2", "value2"))

# Start the threads
thread1.start()
thread2.start()

# Wait for the threads to finish
thread1.join()
thread2.join()

print("Shared dictionary:", shared_dict)


Shared dictionary: {'key1': 'value1', 'key2': 'value2'}


In these examples, mutexes are used to ensure that only one thread at a time can access and modify shared resources like counters, lists, or dictionaries. The acquire() and release() methods of the mutex ensure proper synchronization, preventing race conditions and data corruption. Using context managers (with statements) with mutexes provides a more concise and safe way to manage the acquisition and release of the mutex.

##Multithreading

Life-cycle of a Thread:
  * **Create** by calling the constructor Thread and passing the function to the 'target' keyword along with the arguments (in the 'args' keyword argument of Thread, if the function expects so).
  * **Start** the thread i.e. transition to 'run' state by calling 'start()' on thread object.
  * **Terminated** thread, which happens if the subsequent new thread created by main thread has completed the executio of the program or raised exception.

In [ ]:
from threading import Thread
from time import sleep

def task():
  sleep(1)
  print("Inside task")

def new_task():
  sleep(1)
  print("Inside NEW task")

thread1 = Thread(target=task)
thread1.start()
thread1.join(timeout=2)

thread2 = Thread(target=new_task)
thread2.start()
# thread2.join(timeout=2)
# thread2.join(timeout=0.1)

if thread2.is_alive():
  print("Thread 2 is still running")
else:
  print("Thread 2 is not running")

Inside task
Thread 2 is still running


In the above code, few things that need discussion:
  * Can we call new_task() funtion on thread1 at the same time as we called task()?
    * Nope, it's not possible to call multiple target functions on a single thread. We need to declare separate threads for sifferent functions.

  * What is the purpose of join() on a Thread?
    * join() waits for a thread to complete its execution. This duration of waiting can be restricted by using 'timeout' in join() which allows the later code to run if, for example, the timeout is smaller than sleep time inside the function called on that thread. This is the same effect achieved by not using join(). The results will be same in both the cases as it can be seen from the above output where Thread 2 is still running though it didn't print the statement inside new_task() function. Observe the outputs by commenting line 18 and 19, and then uncomment line 19. However, now comment line 19 and uncomment 18, you will see the difference as why we need join().





---



We use Multithreading for I/O tasks which requires waiting for external events and not for the CPU bound tasks. For that, we use Multiprocessing module from Python. Thread is the smallest unti of program. A process will have at least on thread and so many threads, under one process, share the memory space.

##Multiprocessing

**CPU-bound tasks** are tasks that primarily involve computation and processing within the CPU rather than waiting for input/output operations. These tasks are limited by the processing speed of the CPU and can be compute-intensive, such as mathematical calculations, data processing, complex algorithms, and simulations

The **Global Interpreter Lock (GIL)** in Python can have a significant impact on CPU-bound tasks because it restricts the execution of Python bytecode to a single thread. As a result, even when multiple threads are used for CPU-bound tasks, they won't run in true parallel on multi-core processors due to the GIL.

However, there are strategies to mitigate the impact of the GIL on CPU-bound tasks and potentially speed up their execution:
- Use **Multiprocessing**: Instead of using threads, you can use the multiprocessing module to create separate processes. Each process has its own Python interpreter and memory space, allowing them to run in true parallel, as the GIL doesn't affect processes. This approach leverages multiple CPU cores effectively for CPU-bound tasks.

- Use **Compiled Extensions**: Consider using compiled extensions or libraries written in languages like C or Cython for performance-critical sections of your code. These languages can bypass the GIL and perform computations more efficiently.

- Use **Concurrency Libraries**: Libraries like concurrent.futures provide higher-level interfaces for executing CPU-bound tasks concurrently using threads or processes. They abstract the management of threads and processes, allowing you to focus on the task logic.

- Use **GIL-Free Libraries**: Some Python libraries and tools, like NumPy and pandas, are designed to work efficiently with large arrays and data frames and can bypass the GIL. Utilizing such libraries can speed up computations.

- Consider Using **Other Languages**: For extremely CPU-intensive tasks, you might consider using other programming languages like C, C++, or Rust that don't have a GIL and offer better performance.

It's important to note that while these strategies can help mitigate the impact of the GIL on CPU-bound tasks, the GIL itself is an intrinsic part of CPython (Python's reference implementation), and **it's not possible to completely eliminate** it within the standard Python interpreter.

In [ ]:
import multiprocessing

def process_task():
    # Your CPU-bound task

if __name__ == "__main__":
    num_processes = multiprocessing.cpu_count()
    processes = [multiprocessing.Process(target=process_task) for _ in range(num_processes)]
    for process in processes:
        process.start()
    for process in processes:
        process.join()


Multi-Threading

  - Threads share the same memory and can write to and read from shared variables
  - Due to Python Global Interpreter Lock, two threads won’t be executed at the same time, but concurrently (for example with context switching)
  - Effective for I/O-bound tasks
  - Can be implemented with threading module
  - Multi-processing

Multi-Processing:
  - Every process has is own memory space
  - Every process can contain one ore more subprocesses/threads
  - Can be used to achieve parallelism by taking advantage of multi-core machines since processes can run on different CPU cores
  - Effective for CPU-bound tasks
  - Can be implemented with multiprocessing module (or concurrent.futures.ProcessPoolExecutor)

##Decorators

In [ ]:
# def outer_func(arg_outer_def):
def outer_func():
  def outer_func_2(func):
    def real_func(*args, **kwargs):
      # def inner_func(arg1):
      def inner_func(*args, **kwargs):
        # print("Before calling..", type(arg1))
        print("Before calling..", args, kwargs)
        # arg1 = str(arg1) + arg_outer_def
        # func(arg1)
        func(args)
        # print()
        # return func(arg1)
      return inner_func
    return real_func
  return outer_func_2

# @outer_func(arg_outer_def="static")
@outer_func()
def after_func(arg1="test"):
  # if isinstance(arg1, str):
  print("hELLLOOO")
  if isinstance(arg1, list):
    print("As expected, arg is converted to string: ", type(arg1))
    print(arg1)


# after_func([4,5,6])
after_func([4,5,6], arg1="hello")

<function __main__.outer_func.<locals>.outer_func_2.<locals>.real_func.<locals>.inner_func(*args, **kwargs)>

In [ ]:
def outer_func():
    def inner1(func):
        def inner2(*args, **kwargs):
            def inner3(args):
                def inner4(args):
                    def inner5(args):
                        print("Inside inner5")
                        func(*args, **kwargs)
                        print("Exiting inner5")
                    return inner5
                return inner4
            return inner3
        return inner2
    return inner1

@outer_func()
def decorated_function(arg1="test"):
# def decorated_function(arg1):
    print("Decorated function with arg1:", arg1)

decorated_function("Hello!")


<function __main__.outer_func.<locals>.inner1.<locals>.inner2.<locals>.inner3(args)>

In [ ]:
def outer_func(func):
    def inner1(*args, **kwargs):
        def inner2(*args, **kwargs):
            print("Inside inner2")
            func(*args, **kwargs)
            print("Exiting inner2")
        return inner2
    return inner1

@outer_func
def decorated_function(arg1="test"):
    print("Decorated function with arg1:", arg1)

# Call the decorated function
decorated_function("Hello!")


<function __main__.outer_func.<locals>.inner1.<locals>.inner2(*args, **kwargs)>

In [ ]:
def outer_func(who):
    def inner_func():
        print(f"Hello, {who}")
    inner_func()


outer_func("World!")

Hello, World!


Important points about decorators:
* The signature of the most inner function and the function which is being decorated must be same or with variable args.

What do we mean by something is callable?
- Check the below code which is having outer and inner functions. When the outer function is called it just assigns the object of inner function so that later on the same can be invoked with proper signature of inner function. For example, if we comment line 4 and 5, and uncomment the lines 7,8 and 9, the obj_outer still holds the object of inner_func but if we change the call to a constant instead of a function inside out_function body (as currently it is showing), then you get the error which should be as the string object which is being passed upon call is no more a function but just a constant.

In [ ]:
def out_function(out_args):
  print("hello, outside: ", out_args)

  value = out_args
  return value

  # def inner_func(*args):
  #   return "hello, inside " + str(args) + str(out_args)
  # return inner_func



obj_outer = out_function("_outercall_")
obj_outer("_innercall_")


hello, outside:  _outercall_


TypeError: ignored

The use cases for Python decorators are varied. Here are some of them:

* Debugging
* Caching
* Logging
* Timing

Functools.wraps decorator:

In [ ]:
import functools

def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Decorator is called")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def my_function():
    """This is the docstring of my_function."""
    print("Inside my_function")

print("Name of the decorated function:", my_function.__name__)
print("Docstring of the decorated function:", my_function.__doc__)


Name of the decorated function: wrapper
Docstring of the decorated function: None


In the example above, when we run the code, we'll notice that the name and docstring of the my_function are overwritten by the wrapper function inside the my_decorator.

To preserve the original metadata of the decorated function, we can use functools.wraps. Here's how we can modify the decorator to use functools.wraps:

In [ ]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Decorator is called")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def my_function():
    """This is the docstring of my_function."""
    print("Inside my_function")

print("Name of the decorated function:", my_function.__name__)
print("Docstring of the decorated function:", my_function.__doc__)


Name of the decorated function: my_function
Docstring of the decorated function: This is the docstring of my_function.


By using functools.wraps, we ensure that the decorated function retains its original metadata, making our code more maintainable and easier to understand during debugging and analysis.

##Closures

In [ ]:
def outer_func1():
  sample = []
  def inner_func1(num):
    sample.append(num)
    return sum(sample)/ len(sample)

  return inner_func1



100.0

In [ ]:
obj1 = outer_func1()
obj1(100)

100.0

In [ ]:
# obj1 = outer_func1()
obj1(105)

102.5

Closures are used to save the environmental state of the function with the use of inner fucntion. Here we can see, the list to which elements are being appended is saved and recalled from the last point and used subsequently.

Similalry, with the use of decorator in the below code, we can see this can be maintained and resumed. The value of exponent is not being sent again and again.

In [ ]:
def generate_power(exponent):
  def power(func):
    def inner_power(*args):
      base = func(*args)
      return base**exponent
    return inner_power
  return power


@generate_power(2)
def raise_to_two(n):
  return n

@generate_power(3)
def raise_to_three(n):
  return n

In [ ]:
raise_to_two(5)


25

In [ ]:
raise_to_two(10)


100

In [ ]:
raise_to_three(5)


125

In [ ]:
raise_to_three(6)

216

##ContextManager

Using **decorator** for debug purpose here.

In [ ]:
class MyFileManager:

  def debug(func):
    def wrapper(*args, **kwargs):
      print(f"Calling {func.__name__} with args: {args} {kwargs}")
      result = func(*args, **kwargs)
      print(f"{func.__name__} returned {result}")
      return result
    return wrapper

  @debug
  def __init__(self, filename, mode):
    self.filename = filename
    self.mode = mode

  @debug
  def __enter__(self):
    # print("inside ")
    self.file = open(self.filename, self.mode)
    return self.file

  @debug
  def __exit__(self, exc_type, exc_value, traceback):
  # def __exit__(self):
    if self.file:
      self.file.close()



In [ ]:
with MyFileManager("/content/Capture.txt", 'w') as file:
  file.write("Hello, I am writing")

Calling __init__ with args: (<__main__.MyFileManager object at 0x7bfbf3c408e0>, '/content/Capture.txt', 'w') {}
__init__ returned None
Calling __enter__ with args: (<__main__.MyFileManager object at 0x7bfbf3c408e0>,) {}
__enter__ returned <_io.TextIOWrapper name='/content/Capture.txt' mode='w' encoding='UTF-8'>
Calling __exit__ with args: (<__main__.MyFileManager object at 0x7bfbf3c408e0>, None, None, None) {}
__exit__ returned None


##Generators

Here's how generators work step by step:

Generator Function Definition:
You define a generator function using the yield keyword. When the generator function is called, it doesn't execute immediately but returns a generator object, which acts as an iterator.

First Call to the Generator Function:
When you make the first call to the generator function, its code doesn't run completely. Instead, it runs until it encounters the yield statement. It then suspends its execution and yields the value provided after the yield keyword.

Value Retrieval:
When you request the next value from the generator using the next() function or in a loop, the generator resumes its execution from where it was last suspended. It continues until it hits another yield statement or runs out of code. It then yields the new value and suspends execution again.

Iterating Through the Generator:
You can use a for loop or other iteration methods to retrieve values from the generator. Each time you iterate, the generator produces and yields the next value on-the-fly.

Stop Iteration:
When the generator function runs out of code to execute or encounters a return statement, a StopIteration exception is raised, indicating the end of the iteration.

In [ ]:
def generator_example(limit):
  num = 1
  while num <= limit:
    print("1st example")
    yield num
    num += 1

In [ ]:
ge = generator_example(5)
print(next(ge))
print(next(ge))
print(next(ge))
print(next(ge))
print(next(ge))
# print(next(ge))
# for i in ge:
#   print(i)

1
2
3
4
5


Generator expression:

Just like list comprehension, it stores the values in a variable which is a generator object and then upon using next on this iterable, we can get the values or we can use for loop.

In [ ]:
squares = (x**2 for x in range(1,5))

In [ ]:
next(squares)

4

Keep running the above cell and it will return the values one by one. It means the genrator always resumes from where it was left. Generators provide memory-efficient iteration and are especially useful when dealing with large datasets, streaming data, or scenarios where we want to generate values on-the-fly. They contribute to cleaner and more readable code by avoiding the need to generate and store all values in memory at once.

In [ ]:
for i in squares:
  print(i)

9
16


From the above cell result, it can be observed that since only last two results were left in the squares generator object and hence only those are printed.

In [ ]:
squares = (x**2 for x in range(1,5))

In [ ]:
def gen_squares(sq_gen):
  for i in sq_gen:
    yield i

In [ ]:
for sq in gen_squares(squares):
  print(sq)

1
4
9
16


In [ ]:
def gen_ex2(limit):
  yield from generator_example(limit)
  if limit < 10:
    print("2nd example")
    yield limit


In [ ]:
def iterate_genex(genex):
  if not any(i for i in genex):
    print("The generator is empty, Fill in some fuel")
  else:
    for i in genex:
        print(i)

In [ ]:
genex = gen_ex2(12)

In [ ]:
iterate_genex(genex)

1st example
1st example
2
1st example
3
1st example
4
1st example
5
1st example
6
1st example
7
1st example
8
1st example
9
1st example
10
1st example
11
1st example
12


In [ ]:
genex = gen_ex2(5)

In [ ]:
iterate_genex(genex)

1st example
1st example
2
1st example
3
1st example
4
1st example
5
2nd example
5


In [ ]:
iterate_genex(genex)

The generator is empty, Fill in some fuel


In [ ]:
list(genex)

[]

##With Statement

The with statement in Python is used to simplify the management of resources, such as files, network connections, or locks. It ensures that the resource is properly acquired and released, **even in the presence of exceptions**. Here are some examples of using the with statement with different types of resources:

**Using with for File Handling:**

In [ ]:
# Opening a file using 'with' statement
with open('example.txt', 'r') as file:
    content = file.read()
    print(content)

# File is automatically closed when the 'with' block is exited


**Using with for Locks (Mutexes):**

In [ ]:
import threading

# Mutex for controlling access to shared resource
mutex = threading.Lock()

# Using 'with' statement to acquire and release the mutex
def thread_function():
    with mutex:
        # Critical section
        print("Thread is inside the critical section")

# Create and start a thread
thread = threading.Thread(target=thread_function)
thread.start()
thread.join()


**Using with for Network Connections:**

In [ ]:
import socket

# Creating a socket using 'with' statement
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(('example.com', 80))
    s.send(b'GET / HTTP/1.1\r\n\r\n')
    response = s.recv(4096)
    print(response.decode())
# Socket is automatically closed when the 'with' block is exited


**Using with for Custom Context Managers:**

In [ ]:
class MyContext:
    def __enter__(self):
        print("Entering the context")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting the context")
        if exc_type is not None:
            print(f"Exception occurred: {exc_type}, {exc_value}")
        return False  # Re-raise the exception if needed

# Using custom context manager with 'with' statement
with MyContext() as context:
    print("Inside the context")
    # Uncomment the next line to simulate an exception
    # raise Exception("An error occurred")


##Partial Functions

Partial functions as the name says, is used to freeze a function with arguments which is needed only once and thereafter, you want to use that function everytime with less number of arguments.

This can be useful when you have a function that takes multiple arguments, but you want to create a new function with some of those arguments pre-set, essentially creating a simpler version of the original function.

The functools.partial function is used to create partial functions. It takes a function and its arguments as arguments, and returns a new function with the specified arguments "**frozen**."

Here's a simple use case of partial functions:

In [ ]:
from functools import partial
import random
# Original function
def power(base, exponent):
    # base=1
    return base ** exponent

cubes = partial(power, exponent=3)
cubical_results = []
for _ in range(10):
  j = random.randint(1, 5)
  cubical_results.append(cubes(j))

cubical_results

[8, 27, 8, 8, 8, 27, 1, 8, 64, 125]

In this example, the cubes function is a partial function created from the power function, with the base argument fixed to 3.

Another use case for partial functions is when working with **callback functions** that require specific arguments. For example, in GUI programming or event handling, you might need to pass additional arguments to a callback function:

In [ ]:
import functools

# def on_button_click(event, button_id, button_name, button_val):
def on_button_click(event, button_name, button_val, button_id):
    print(f"Button {button_id} clicked!")

# Create partial functions for specific button IDs
on_button_1_click = functools.partial(on_button_click, button_val=4, button_id=1)
# on_button_1_click = functools.partial(on_button_click, button_val=4)
on_button_2_click = functools.partial(on_button_click, button_id=2)

# Simulate button clicks
on_button_1_click("Click event", "REX")  # Output: Button 1 clicked!
on_button_2_click("Click event", "REX more", 8)  # Output: Button 2 clicked!


Button 1 clicked!
Button 2 clicked!


Few points to note here:
* Once the function is frozen with the given arguments, you cannot pass that same argument again.
* Another thing is, the passes arguments while the function is being frozen (line 8 and 9), the arguments for which values are being sent to the functions should start from end and not in the middle as shown in the commented line 3.

Partial functions are particularly useful when you have functions that take many arguments but you frequently need to use them with certain arguments held constant. They can help improve code readability and reduce the complexity of function calls.

##Lambda Functions

Lambda functions in Python are used for creating small, anonymous functions without needing to define them using the def keyword. Here are some examples of different ways you can use lambda functions:

**Sorting with Lambda:**

In [ ]:
list_of_tuples = [(4,5), (5,10), (0,6), (11,7)]

sorted_tuples = sorted(list_of_tuples, key=lambda i: i[1])
sorted_tuples

[(4, 5), (0, 6), (11, 7), (5, 10)]

**Using Lambda in Map:**

In [ ]:
numbers = [1, 2, 3, 4, 5]
squared_numbers = map(lambda x: x ** 2, numbers)

print(list(squared_numbers))  # Output: [1, 4, 9, 16, 25]


**Using Lambda in Filter:**

In [ ]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9]
even_numbers = filter(lambda x: x % 2 == 0, numbers)

print(list(even_numbers))  # Output: [2, 4, 6, 8]


**Using Lambda in Reduce:**

In [ ]:
from functools import reduce

numbers = [1, 2, 3, 4, 5]
product = reduce(lambda x, y: x * y, numbers)

print(product)  # Output: 120


**Using Lambda in Custom Functions:**

In [ ]:
def apply_operation(lambda_func, x, y):
  return lambda_func(x,y)

output1 = apply_operation(lambda k,l : "sum: " + str(k+l) , 5, 6)
output2 = apply_operation(lambda kk,ll : "multiply: " + str(kk*ll) , 5, 6)

output1, output2

('sum: 11', 'multiply: 30')

##Descriptors

Descriptors are a powerful mechanism to give custom behavior to attribute access. Descriptors are objects that define ***at least one*** of the special methods: `__get__(), __set__(),` or `__delete__()`. When an object's attribute is accessed, these methods can control the behavior of the attribute.

Here's a quick rundown:

* `__get__`(self, instance, owner):

  Used to retrieve the value of an attribute.
  instance is the instance for which the attribute was accessed (or None if accessed on the class itself).
  owner is the owner class of the descriptor.
* `__set__`(self, instance, value):

  Called when we try to assign a value to the attribute.
  instance is the instance to which the value is assigned.
  value is the value to be assigned.
* `__delete__`(self, instance):

  Invoked when del is used to delete the attribute from an instance.
  instance is the instance from which the attribute should be deleted.
* `__set_name__`(self, owner, name):

  Available from Python 3.6 onward.
  Automatically called at the time of class creation.
  Allows the descriptor to know both the class it is in (owner) and the name of the attribute (name).
  Beyond these core descriptor methods, there aren't other "built-in" descriptor methods in Python. However, other methods and attributes can be defined on descriptor classes as needed, but they won't have the special behavior that the core descriptor methods have.

That said, the presence or absence of these methods affects how the descriptor behaves:

A descriptor that only implements `__get__` is called a "**non-data descriptor**" or "non-overriding descriptor". It cannot manage setting or deleting, and it can be easily overridden by instance attributes.

A descriptor that implements `__set__` or `__delete__` (or both) is a "**data descriptor**" or "overriding descriptor". Data descriptors have precedence over instance attributes, meaning that if you have a data descriptor named foo and an instance attribute named foo, the descriptor will "win" in attribute lookups and the instance attribute will be effectively hidden.

Descriptor Protocol:

`__get__(self, obj, type=None)`: This method returns the value of the attribute for a specific object (obj). The type argument is optional and if provided, is the type of the obj.

`__set__(self, obj, value)`: This method is used to set the value of the attribute for a specific object (obj).

`__delete__(self, obj)`: This method is used to delete an attribute from an object.

Descriptors are often used for:

* **Data validation**: Ensure that values assigned to an attribute meet certain conditions.

* **Property calculation**: Compute the value of a property dynamically when it's accessed.

* **Logging or tracking**: Log access to a certain attribute.

* **Lazy attributes**: Compute the value of an attribute the first time it's accessed and store it for future use.

* **Object-relational mapping**: Mapping database records to Python objects.